In [1]:
import django
import pandas as pd
import requests
django.setup()

import os
from pprint import pprint
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from sch.models import *
import datetime as dt
from django.db.models import CharField
from django.db.models.functions import Cast
from django.test import RequestFactory
from sch.actions import *
from sch.views2 import *
from django.db.models import Sum

# Weekly UnderHours Calculations

In [20]:
class WeeklyUnderHoursCalcs:

    def __init__(self, schid):
        self.schid = schid
        self.sch = Schedule.objects.get(slug=schid)
        self.employees = self.sch.employees.all()

    def __post_init__(self):
        self.employees_weekly_hours = self.all_empl_week_sum()
    
        

    def empl_week_sum (self, empl):
        employee = Employee.objects.get(slug=empl)
        sch = Schedule.objects.get(slug=self.schid)

        wksums = []
        for wk in sch.weeks.all():
            wk.slots.filter(employee=employee).aggregate(Sum('shift__hours'))
            wksums.append(wk.slots.filter(employee=employee).aggregate(Sum('shift__hours'))['shift__hours__sum'])
        return {employee.slug: wksums}


    def all_empl_week_sum (self):
        sch = Schedule.objects.get(slug=self.schid)
        empls = sch.employees.all()
        empl_wksums = {}
        for empl in empls:
            empl_wksums.update(self.empl_week_sum(empl.slug))
        return empl_wksums
    

    def get_sch_empl_maxes (self):
        sch = Schedule.objects.get(slug=schid)

        all_empl_maxes = {}
        for empl in sch.employees.all():
            if empl.slug in sch.data['maxes']:
                all_empl_maxes.update({empl.slug: int(sch.data['maxes'][empl.slug])})
            else:
                all_empl_maxes.update({empl.slug: empl.std_wk_max})

        for empl in all_empl_maxes:
            empl = Employee.objects.get(slug=empl)
            if empl not in sch.data['maxes']:
                all_empl_maxes.update({empl.slug: empl.std_wk_max })
        return all_empl_maxes



In [21]:
undertimes = WeeklyUnderHoursCalcs('2023-S4A')
undertimes.employees_weekly_hours()

TypeError: 'dict' object is not callable

In [7]:

def get_sch_empl_needed_hours (schid):
    from typing import Dict, List
    sch = Schedule.objects.get(slug=schid)

    empl_maxes = get_sch_empl_maxes(schid) # type: Dict[str, list]
    empl_wksums = all_empl_week_sum(schid) # type: Dict[str, int]
    

    empl_needed_hours = {} 
    for empl in sch.employees.all():
        empl_list = []
        for wk in empl_wksums[empl.slug]:
            if wk == None:
                wk = 0
                empl_list.append(empl_maxes[empl.slug] - wk)
            else:
                empl_list.append(empl_maxes[empl.slug] - wk)
        empl_needed_hours.update({empl.slug: empl_list})
    

    return empl_needed_hours


get_sch_empl_needed_hours('2023-S4A')


NameError: name 'get_sch_empl_maxes' is not defined

In [9]:
def schedule__get_undertime_hours(request, schId):
   """
   example output:
   >>> {
      "employee":     "Brianna-A", 
      "under_hours":  10.0, 
      "wk1_hours":    30.0,
      "wk2_hours":    40.0,
      "potentials":   [["EP", "2023-05-10C"]]
   }
   """
   sch = Schedule.objects.get(slug=schId)

   uts = []
   for pd in sch.periods.all():
      pd_ut = []
      pd.save()
      for empl in sch.employees.filter(fte__gt=0):
            ut = {}
            potential_fill_list = []

            pd_hours = pd.slots.filter(employee=empl).aggregate(Sum("shift__hours"))['shift__hours__sum'] or 0
            pto_hours = PtoRequest.objects.filter(employee=empl,workday__in=pd.workdays.values('date')).count() * 10
            
            w1sd_ids = [i for i in range(42) if i//7 in [0,2,4]]
            wdsd_ids = [i for i in range(42) if i//7 in [1,3,5]]
            
            w1hrs = pd.slots.filter(workday__sd_id__in=w1sd_ids, employee=empl).aggregate(Sum("shift__hours"))['shift__hours__sum'] or 0
            w2hrs = pd.slots.filter(workday__sd_id__in=wdsd_ids, employee=empl).aggregate(Sum("shift__hours"))['shift__hours__sum'] or 0
            w1pto = PtoRequest.objects.filter(
               employee=empl,
               workday__in=pd.slots.filter(workday__sd_id__in=w1sd_ids).values('workday__date')
               ).count() * 10
            w2pto = PtoRequest.objects.filter(
               employee=empl,
               workday__in=pd.slots.filter(workday__sd_id__in=wdsd_ids).values('workday__date')
               ).count() * 10

            if pd_hours < empl.fte * 80:
               total = empl.fte * 80 - pd_hours - pto_hours 
               if total > 0:
                  pots = pd.slots.empty().filter(fills_with=empl).exclude(workday__slots__employee=empl)
                  for pot in pots:
                        if pot:
                           if not pot.actions.check_turnaround_risk(pot, empl):
                              potential_fill_list += [(pot.shift.name,pot.workday.slug, pot.workday.week.number%2)] 
                  
                  ut['employee']= empl.slug 
                  ut['under_hours']= total
                  if (empl.fte * 40 ) % 10 == 0:
                        ut['wk1_hours']= empl.fte * 40 - (w1hrs - w1pto)
                        ut['wk2_hours']= empl.fte * 40 - (w2hrs - w2pto)
                  else:
                        heavy_week = random.choice([0,1])
                        if heavy_week == 0:
                           ut['wk1_hours']= empl.fte * 40 - (w1hrs - w1pto) + 5
                           ut['wk2_hours']= empl.fte * 40 - (w2hrs - w2pto) - 5
                        else:
                           ut['wk1_hours']= empl.fte * 40 - (w1hrs - w1pto) - 5
                           ut['wk2_hours']= empl.fte * 40 - (w2hrs - w2pto) + 5
                  
                  ut['potentials'] = potential_fill_list
                  pd_ut += [ut]

      uts += [pd_ut]
   return uts


schedule__get_undertime_hours(None, '2023-S4B')

[[{'employee': 'Cheryl',
   'under_hours': 10.0,
   'wk1_hours': 0.0,
   'wk2_hours': 50.0,
   'potentials': [('EP', '2023-06-22B', 1)]},
  {'employee': 'David',
   'under_hours': 30.0,
   'wk1_hours': 20.0,
   'wk2_hours': 10.0,
   'potentials': [('S', '2023-06-15B', 0), ('EI', '2023-06-15B', 0)]},
  {'employee': 'Jaden',
   'under_hours': 30.0,
   'wk1_hours': 10.0,
   'wk2_hours': 20.0,
   'potentials': [('S', '2023-06-15B', 0), ('EP', '2023-06-22B', 1)]},
  {'employee': 'Mandy',
   'under_hours': 20.0,
   'wk1_hours': 10.0,
   'wk2_hours': 10.0,
   'potentials': []},
  {'employee': 'Teddy',
   'under_hours': 40.0,
   'wk1_hours': 30.0,
   'wk2_hours': 10.0,
   'potentials': [('EI', '2023-06-15B', 0)]}],
 [{'employee': 'Cheryl',
   'under_hours': 20.0,
   'wk1_hours': 20.0,
   'wk2_hours': 0.0,
   'potentials': [('EP', '2023-06-26B', 0), ('EI', '2023-07-06B', 1)]},
  {'employee': 'David',
   'under_hours': 10.0,
   'wk1_hours': 10.0,
   'wk2_hours': 0.0,
   'potentials': [('MI', '20

In [56]:
def schedule__get_undertime_hours(request, schId ):
   sch = Schedule.objects.get(slug='2023-S4B')
   empl_dict = {empl.slug:{'weeks':[],'periods':[],'fte':empl.fte} for empl in sch.employees.filter(fte__gt=0)}
   wk_empl_dict = {empl.slug:[] for empl in sch.employees.all()}

   for pd in sch.periods.all():
      for week in pd.weeks.all():
            for empl in sch.employees.all():
               if week.slots.filter(employee=empl).count() > 0:
                  slot_sum = sum(list(week.slots.filter(employee=empl).values_list('shift__hours',flat=True)))
                  pto_sum  = week.pto_requests().filter(employee=empl).count() * 10
                  wk_empl_dict[empl.slug] += [slot_sum + pto_sum]
               else:
                  wk_empl_dict[empl.slug] += [0]

   pd_empl_dict = {empl.slug:[] for empl in sch.employees.all()}
   for empl in wk_empl_dict:
      pd_empl_dict[empl] += [wk_empl_dict[empl][0] + wk_empl_dict[empl][1]]
      pd_empl_dict[empl] += [wk_empl_dict[empl][2] + wk_empl_dict[empl][3]]
      pd_empl_dict[empl] += [wk_empl_dict[empl][4] + wk_empl_dict[empl][5]]
   wk_empl_dict

   for emp in empl_dict:
      empl_dict[emp]['weeks'] = wk_empl_dict[emp]
      empl_dict[emp]['periods'] = pd_empl_dict[emp]
      
      i = 0
      for wk in empl_dict[emp]['weeks']:
         if wk < empl_dict[emp]['fte'] * 40:
            if empl_dict[emp]['periods'][i//2] < empl_dict[emp]['fte'] * 80:
               empl_dict[emp][f"needed-week-{i}"] = empl_dict[emp]['fte'] * 40 - empl_dict[emp]['weeks'][i]
               empl_dict[emp][f"week-{i}-empty"] = list(sch.weeks.all()[i].slots.filter(
                                                      employee__isnull=True, 
                                                      fills_with__slug=emp)
                                                      .values_list('slug',flat=True))
               empl_dict[emp][f"week-{i}-empty"] = [j for j in empl_dict[emp][f"week-{i}-empty"] if j is not None]
               empl_dict[emp][f"week-{i}-steal"] = list(sch.weeks.all()[i].slots.filter(
                                                      employee__fte=0,
                                                      fills_with__slug=emp)
                                                      .values_list('slug',flat=True))
               empl_dict[emp][f"week-{i}-steal"] = [j for j in empl_dict[emp][f"week-{i}-steal"] if j is not None]
         i += 1
   return empl_dict # type: Dict[str, Dict[str, Any]]

schedule__get_undertime_hours(None, '2023-S4B')

{'Amanda': {'weeks': [40.0, 20.0, 40.0, 30.0, 40.0, 30.0],
  'periods': [60.0, 70.0, 70.0],
  'fte': 0.875,
  'needed-week-1': 15.0,
  'week-1-empty': [],
  'week-1-steal': []},
 'Brianna-A': {'weeks': [40.0, 40.0, 30.0, 40.0, 40.0, 40.0],
  'periods': [80.0, 70.0, 80.0],
  'fte': 1.0,
  'needed-week-2': 10.0,
  'week-2-empty': [],
  'week-2-steal': ['2023-06-26B-MI']},
 'Brittanie': {'weeks': [40.0, 40.0, 40.0, 40.0, 40.0, 40.0],
  'periods': [80.0, 80.0, 80.0],
  'fte': 1.0},
 'Cheryl': {'weeks': [40.0, 10.0, 20.0, 40.0, 40.0, 30.0],
  'periods': [50.0, 60.0, 70.0],
  'fte': 1.0,
  'needed-week-1': 30.0,
  'week-1-empty': ['2023-06-22B-EP'],
  'week-1-steal': ['2023-06-20B-7C'],
  'needed-week-2': 20.0,
  'week-2-empty': ['2023-06-26B-EP'],
  'week-2-steal': ['2023-06-26B-MI'],
  'needed-week-5': 10.0,
  'week-5-empty': ['2023-07-18B-EI'],
  'week-5-steal': ['2023-07-16B-EI',
   '2023-07-17B-MI',
   '2023-07-17B-7C',
   '2023-07-18B-MI']},
 'Danica': {'weeks': [40.0, 40.0, 40.0, 40.0

In [7]:
def annotate_workdays_with_employee_status(schedule, employee):
   
   if isinstance(schedule, str):
      schedule = Schedule.objects.get(slug=schedule)
   if isinstance(employee, str):
      employee = Employee.objects.get(name=employee)

   workdays = schedule.workdays.all()  # Assuming a ForeignKey relationship between Schedule and Workday

   # Annotate workdays with employee slots
   workdays = workdays.annotate(
      employee_slot=Coalesce(
         Case(
               When(slots__employee=employee, then='slots__id'),
               default=None,
               output_field=CharField()
         ),
         Value(''),
      )
   )

   # Annotate workdays with employee PTO
   workdays = workdays.annotate(
      employee_pto=Coalesce(
         Case(
               When(pto_requests__employee=employee, then='pto_requests__id'),
               default=None,
               output_field=CharField()
         ),
         Value(''),
      )
   )

   # Annotate workdays with employee_status
   workdays = workdays.annotate(
      employee_status=Case(
         When(employee_slot='', then=Value('No Slot')),
         When(employee_pto='',  then=Value('Slot assigned')),
         default=Value('PTO'),
         output_field=CharField()
      )
   )

   return workdays

In [50]:
from typing import Tuple

def get_period_triplet_for_employee(schid:Schedule|str, empid:Employee|str) -> Tuple[int,int,int]:
   """A Period Triplet is a set of 3 ints,
   each representing the number of hours the 
   employee is underscheduled.
   
   ex: 
   >>> Josh (0,0,20)      means Josh is underscheduled 20 hours during the 3rd pay period.
   >>> Brittanie (0,0,0)  means Brittanie is fully scheduled.
   """

   empl = Employee.objects.get(slug=empid)
   sch = Schedule.objects.get(slug=schid)
   
   triplet = ()

   for p in range(3):
      phours = empl.fte * 80
      for w in range(2):
         for wd in sch.periods.all()[p].weeks.all()[w].workdays.all():
            if wd.slots.filter(employee=empl).count() > 0:
               phours -= wd.slots.filter(employee=empl).first().shift.hours
            elif wd.pto_requests.filter(employee=empl).count() > 0:
               phours -= empl.pto_hrs
      if phours < 0:
         phours = 0
         
      triplet += (int(phours),)
      
   return triplet
   
pd_triplet = lambda sch, emp : get_period_triplet_for_employee(sch, emp)


In [52]:
sch.employees.all()

<EmployeeManager [<Employee: Amanda>, <Employee: Blake>, <Employee: Brianna A>, <Employee: Brittanie>, <Employee: Cheryl>, <Employee: Danica>, <Employee: David>, <Employee: Elisia>, <Employee: Esperanza>, <Employee: Jaden>, <Employee: Josh>, <Employee: Leslie>, <Employee: Lindsay>, <Employee: Mandy>, <Employee: Mark>, <Employee: Michael>, <Employee: Mona>, <Employee: Nick>, <Employee: Sabrina>, <Employee: Teddy>, '...(remaining elements truncated)...']>

# Takeable Slots

In [2]:
sch = Schedule.objects.all()[6]

def takeable (self):

   takeables = []
   for slot in self.filled():
      week = slot.week
      hours = week.slots.filter(employee=slot.employee).aggregate(Sum('shift__hours'))['shift__hours__sum']
      if hours < 40:
         takeables += [slot.pk]
   return self.filter(pk__in=takeables)


# Undertime Schemas

In [14]:
from schedule.utils import get_undertime_triplet
import yaml
from pprint import pprint

def get_fills_with(schId,empId):
   
   schedule = Schedule.objects.get(slug=schId)
   employee = Employee.objects.get(slug=empId)
   triplet = get_undertime_triplet(schId,empId)
   
   fills_with_data = [[],[],[]]
   schedule_strs = []
   
   for prd in range(len(triplet)):
      if triplet[prd] != 0:
         fills_with = schedule.periods.all()[prd].slots.filter(fills_with=employee)
         fills_with_data[prd] = list(set(fills_with.values_list('slug',flat=True)))
      else:
         fills_with_data[prd] = []
         
      schstr = ""
      for wd in schedule.periods.all()[prd].workdays.all():
         if wd.slots.filter(employee=employee).count() > 0:
            schstr += wd.slots.filter(employee=employee).first().shift.name
            schstr += " "
         else:
            schstr += ". "
      schedule_strs += [schstr]

         
   data = f"""
   schedule: 
      _id: {schedule.slug}
      periods:
         - 1:
            hours_needed: {triplet[0]}
            fills: {fills_with_data[0]}
            schedule: {schedule_strs[0]}
         
         - 2:
            hours_needed: {triplet[1]}
            fills: {fills_with_data[1]}
            schedule: {schedule_strs[1]}
         
         - 3:
            hours_needed: {triplet[2]}
            fills: {fills_with_data[2]}
            schedule: {schedule_strs[2]}
   """
   return yaml.load(data,Loader=yaml.FullLoader)

In [15]:
d = get_fills_with('2023-S4E','Cheryl')

pprint(d)

{'schedule': {'_id': '2023-S4E',
              'periods': [{1: {'fills': ['2023-06-12E-EI',
                                         '2023-06-21E-MI',
                                         'None',
                                         '2023-06-23E-EP',
                                         '2023-06-14E-EI'],
                               'hours_needed': 30,
                               'schedule': '. . . . . 7C EP EI . . . 7C EP .'}},
                          {2: {'fills': ['2023-07-3E-EP',
                                         '2023-07-7E-EP',
                                         '2023-06-28E-N',
                                         '2023-07-3E-EI',
                                         'None',
                                         '2023-07-4E-7C',
                                         '2023-06-26E-EI',
                                         '2023-07-6E-7C',
                                         '2023-06-26E-MI',
                                  

# Constraints

In [4]:
from optapy import problem_fact, planning_variable, planning_entity

In [132]:
from optapy import problem_fact, planning_entity, planning_variable, planning_solution, planning_id


@problem_fact
class Employee:
    def __init__(self, id, name):
        self.id = id
        self.name = name
        
    @planning_id
    def get_id(self): return self.id
    
        
    def __repr__(self): return str(self.name)

@problem_fact
class Workday:
    def __init__(self, id, date):
        self.id = id
        self.date = date
        
    @planning_id
    def get_id(self): return self.id
    
    def __repr__(self): return str(self.date.strftime("%Y-%m-%d"))

@problem_fact
class Timeslot:
    def __init__(self, id, workday, start_time, end_time):
        self.id = id
        self.workday = workday
        self.start_time = start_time
        self.end_time = end_time
    
    @planning_id
    def get_id(self): return self.id
    
    def __repr__(self): return str(self.start_time.strftime("%H:%M"))

@planning_entity
class Shift:
    def __init__(self, id, workday, timeslot, employee=None):
        self.id = id
        self.workday = workday
        self.timeslot = timeslot
        self.employee = employee
        
    @planning_id
    def get_id(self): return self.id
        
    def __repr__(self): return str(self.workday) + " " + str(self.timeslot) + " " + str(self.employee)

    @planning_variable(Employee, ["employeeRange"])
    def get_employee(self):
        return self.employee

    def set_employee(self, employee):
        self.employee = employee
        
    @planning_variable(Workday, ["workdayRange"])
    def get_workday(self):
        return self.workday
    
    def set_workday(self, workday):
        self.workday = workday
        
    @planning_variable(Timeslot, ["timeslotRange"])
    def get_timeslot(self):
        return self.timeslot
    
    def set_timeslot(self, timeslot):
        self.timeslot = timeslot
        
    


In [133]:
from optapy import constraint_provider, get_class
from optapy.types import Joiners, ConstraintFactory
import datetime as dt
from pprint import pprint

# Define the constraint factory
@constraint_provider
def define_constraints(constraint_factory: ConstraintFactory):
    # Get the class references
    Employee = get_class('Employee')
    Shift = get_class('Shift')

    # Constraint 1: Assign each shift to one employee only
    def one_employee_per_shift(constraint_factory: ConstraintFactory):
        return constraint_factory \
            .from_(Shift) \
            .filter(lambda shift: shift.employee is None) \
            .penalize("One employee per shift", 1)

    # Constraint 2: Do not assign more than one shift to an employee on the same day
    def no_more_than_one_shift_per_day(constraint_factory: ConstraintFactory):
        return constraint_factory \
            .from_(Shift) \
            .join(Shift, Joiners.equal(lambda shift: shift.employee),
                          Joiners.equal(lambda shift: shift.workday)) \
            .if_exists(Employee, Joiners.equal(lambda shift: shift.employee)) \
            .penalize("No more than one shift per day", 1)

    # Constraint 3: Do not assign overlapping shifts to the same employee
    def no_overlapping_shifts(constraint_factory: ConstraintFactory):
        return constraint_factory \
            .from_(Shift) \
            .join(Shift, Joiners.equal(lambda shift: shift.employee),
                          Joiners.lessThan(lambda shift: shift.timeslot.start_time,
                                           lambda other: other.timeslot.end_time),
                          Joiners.greaterThan(lambda shift: shift.timeslot.end_time,
                                              lambda other: other.timeslot.start_time)) \
            .if_exists(Employee, Joiners.equal(lambda shift: shift.employee)) \
            .penalize("No overlapping shifts", 1)

    # Add the constraints to the constraint factory
    return [
        one_employee_per_shift(constraint_factory),
        no_more_than_one_shift_per_day(constraint_factory),
        no_overlapping_shifts(constraint_factory)
    ]



In [134]:
from optapy import planning_solution, planning_entity_collection_property, \
                   problem_fact_collection_property, \
                   value_range_provider, planning_score
from optapy.score import HardSoftScore

@planning_solution
class ShiftSchedule:
    def __init__(self, employee_list, workday_list, timeslot_list, shift_list, score=None):
        self.employee_list = employee_list
        self.workday_list = workday_list
        self.timeslot_list = timeslot_list
        self.shift_list = shift_list
        self.score = score
        
    @problem_fact_collection_property(Employee)
    @value_range_provider("employeeRange")
    def get_employee_list(self):
        return self.employee_list

    @problem_fact_collection_property(Workday)
    @value_range_provider("workdayRange")
    def get_workday_list(self):
        return self.workday_list

    @problem_fact_collection_property(Timeslot)
    @value_range_provider("timeslotRange")
    def get_timeslot_list(self):
        return self.timeslot_list

    @planning_entity_collection_property(Shift)
    def get_shift_list(self):
        return self.shift_list

    @planning_score(HardSoftScore)
    def get_score(self):
        return self.score

    def set_score(self, score):
        self.score = score
    
    def __repr__(self):
        return str(self.__dict__)
    


In [135]:
def generate_problem():
    employees = [
        Employee(1, "Alice"),
        Employee(2, "Bob"),
        Employee(3, "Carol")
    ]

    workdays = [
        Workday(1, dt.date(2023, 5, 9)),
        Workday(2, dt.date(2023, 5, 10)),
        Workday(3, dt.date(2023, 5, 11)),
        Workday(4, dt.date(2023, 5, 12)),
        Workday(5, dt.date(2023, 5, 13)),
        Workday(6, dt.date(2023, 5, 14)),
        Workday(7, dt.date(2023, 5, 15)),
        Workday(8, dt.date(2023, 5, 16)),
        Workday(9, dt.date(2023, 5, 17)),
    ]

    timeslots = [
        Timeslot(1, dt.time(7, 0),  dt.time(17, 30)),
        Timeslot(2, dt.time(12, 30), dt.time(23, 0)),
        Timeslot(3, dt.time(21, 0), dt.time(7, 30))
    ]
    
    shifts = []
    shift_id = 1
    for workday in workdays:
        for timeslot in timeslots:
            shifts.append(Shift(shift_id, workday, timeslot))
            shift_id += 1
    
    return ShiftSchedule(employees, workdays, timeslots, [])


In [139]:
import optapy.config
from optapy.types import Duration

solver_config = optapy.config.solver.SolverConfig() \
    .withEntityClasses(Shift) \
    .withSolutionClass(ShiftSchedule) \
    .withConstraintProviderClass(define_constraints) \
    .withTerminationSpentLimit(Duration.ofSeconds(90))